In [4]:
import os
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta
import re

pages = 1
BASE_URL = "https://news.abplive.com/"
country = "bangladesh"
initial_url = f"{BASE_URL}/search/page-{pages}?s={country}"

#FINDING THE TOTAL NUMBER OF PAGES

# pages = 1
# while True:

#     url = f"{BASE_URL}/search/page-{pages}?s={country}"

#     response = requests.get(url)
#     soup = BeautifulSoup(response.text, 'html.parser')

#     arrow_div = soup.find('div',class_ = 'category-pagination')

#     next_page_button = arrow_div.find('a', {'title': 'Next Page'})

#     if next_page_button.get('href') == 'javascript:void(0)':

#         break

#     else:

#         pages+=1
    
# print(f'Total Number of Pages Available is {pages}') 

#SCRAPING NEWS LINKS FROM THE PAGES

news_url_links = []

for page in range(1,3):

    url = f'https://news.abplive.com/search/page-{page}?s=bangladesh'

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    main_link_div = soup.find('div', class_ = 'search-cat-wrap')
    all_links = main_link_div.find_all('a')

    for link in all_links:

        news_url_links.append(link.get('href'))


#SCRAPING NEWS FROM THE LINKS

data_list = []
counter = 0


for link in news_url_links[:20]:

    response = requests.get(link)
    soup = BeautifulSoup(response.text, 'html.parser')

    #TITLE
    
    #THERE ARE TWO DIFFERENT LAYOUTS THAT I'VE SEEN SO FAR IN THE WEBSITE. 
    
    #THERE'S NO FIXED LAYOUT FOR ANY PARTICULAR NICHE.
    
    #So, we check for each type of layout. One is termed as 'abp' and the other is termed as 'no_abp', depending on the layout.
    
    title_div = soup.find('div', {'id':'full-content'})

    if title_div:

        title = title_div.find('h1', class_ = 'article-title').text

        article_type = 'no_abp'

    else:

        main_section = soup.find('section',class_= 'abp-article')

        if main_section:

            title = main_section.find('h1', class_= 'abp-article-title').text
            title = re.sub('/n|/r','',title)
            title = re.sub(r'\xa0', '', title)
            title = title.strip()

            article_type = 'abp'

        else:

            title = 'Title not found'
    
    title_translation = 'None'

    #AUTHOR

    if article_type == 'no_abp':

        author_element = title_div.find('p', class_ = 'article-author')
        author = author_element.find('a').text


    elif article_type == 'abp':

        author_element = main_section.find('div', class_ = 'abp-article-byline-author')
        author = author_element.find('a').text


    else:
        author = 'Author not found'


    #DATE

    date_div = soup.find('meta', {'property': 'article:modified_time'})


    if date_div:

            date_data = date_div.get('content')

            only_date = date_data.split('T')[0]
            only_time = date_data.split('T')[1]
            time = f"{only_time.split(':')[0]}:{only_time.split(':')[1]}"
            cleaned_date = f"{only_date},{time}"

            source_localtime = datetime.strptime(cleaned_date, "%Y-%m-%d,%H:%M") + timedelta(hours = 5, minutes = 30) 
            bangladesh_localtime = source_localtime + timedelta(minutes=30)
    
    else:
        
            date_data = 'Date Data Not Found'



    #CONTENT

    content = []
    
    if article_type == 'no_abp':

        content_div = title_div.find('div', class_ = 'article-data _thumbBrk uk-text-break')
        all_paras = content_div.find_all('p')

        for para in all_paras:

            content.append(para.text)

        full_content = " ".join(content)
        full_content = re.sub('\n|\r','', full_content)
        full_content = re.sub(r'\xa0', '', full_content)
        full_content = full_content.strip()

    elif article_type == 'abp':

        content_div = main_section.find('div', {'id':'article-hstick-inner'})
        all_paras = content_div.find_all('p')

        for para in all_paras:

            content.append(para.text)

        full_content = " ".join(content)
        full_content = re.sub('\n|\r','', full_content)
        full_content = re.sub(r'\xa0', '', full_content)
        full_content = full_content.strip()

    else:
        full_content = 'Content Not Found'

    
    content_translation = 'None'
    #CONTENT SUMMARY

    if article_type == 'no_abp':

        content_summary_tag = title_div.find('h2', class_ = 'article-excerpt')
        content_summary = content_summary_tag.text if content_summary_tag else 'Content summary not found'

    elif article_type == 'abp':

        content_summary_tag = main_section.find('h2',class_= 'abp-article-slug')
        content_summary = content_summary_tag.text if content_summary_tag else 'Content summary not found'

    else:
        content_summary = 'Content summary not found'

    
    summary_translation = 'None'
    
#     data_dict = {
#                 "country": country,
#                 "author": author,
#                 "url": link,
#                 "title": title,
#                 "timestamp": timestamp,
#                 "date": date,
#                 "date_str": date_str,
#                 "content": full_content,
#                 "content_summary": content_summary
#             }
    
    data_dict = {
            "url": link,
            "title": title,
            "content": full_content,
            "content_summary": content_summary,
            "title_translation":title_translation,
            "content_translation":content_translation,
            "summary translation":summary_translation,
            "author": author,
            "country": country,
            'source_localtime': source_localtime,
            'bangladesh_localtime': bangladesh_localtime

        }

    counter+=1


    if (date_data != "Date Data Not Found" and full_content != "Content Not Found" and content_summary != 'Content summary not found' ):
                
            if data_dict not in data_list:
                    # Adding to data list
                    data_list.append(data_dict)
                    print(f'Link {counter} added')
    else:
            print(f'Link {counter}')
            print('Skipped due to missing info.')

df = pd.DataFrame(data_list)
df.head()

csv_filename = f"{country}_ABP_Live.csv"

# Checking if the CSV file already exists
if os.path.exists(csv_filename):
    existing_df = pd.read_csv(csv_filename)
    # Merging new and existing dataframe
    df = pd.concat([existing_df, pd.DataFrame(data_list)], ignore_index=True)
    df["date"] = pd.to_datetime(df["bangladesh_localtime"])  # Converting the "date" column to datetime
    df = df.drop_duplicates(subset=["title"], keep="first")
    df = df.sort_values(by="bangladesh_localtime", ascending=False)  # Sorting the date
    df = df.reset_index(drop=True)
    df.to_csv(csv_filename, index=False)  
else:
    # If csv file does not exist, then we create a new CSV file with the scraped data
    df = pd.DataFrame(data_list)
    df["date"] = pd.to_datetime(df["bangladesh_localtime"], format = "%d-%m-%Y")  # Converting the "date" column to datetime
    df = df.sort_values(by="bangladesh_localtime", ascending=False)
    df = df.drop_duplicates(subset=["title"], keep="first")
    df = df.reset_index(drop=True)
    df.to_csv(csv_filename, index=False)

Link 1 added
Link 2 added
Link 3 added
Link 4 added
Link 5 added
Link 6 added
Link 7 added
Link 8 added
Link 9 added
Link 10 added
Link 11 added
Link 12 added
Link 13 added
Link 14 added
Link 15 added
Link 16 added
Link 17 added
Link 18 added
Link 19 added
Link 20 added


url  \
0  https://news.abplive.com/sports/cricket/bpl-20...   
1  https://news.abplive.com/sports/cricket/bpl-20...   
2  https://news.abplive.com/news/india/citizenshi...   
3  https://news.abplive.com/sports/cricket/bpl-20...   
4  https://news.abplive.com/sports/cricket/bangla...   

                                               title  \
0  BPL 2024 Final Comilla Victorians vs Fortune B...   
1  BPL 2024: Top Run-Scorers, Wicket-Takers In Ba...   
2  Ministry Of Home Affairs To Notify CAA Rules B...   
3  BPL 2024: Highest Run-Scorers, Wicket-Takers I...   
4  Bangladesh Premier League 2024: Updated Points...   

                                             content  \
0  BPL 2024 Final live streaming, playing 11, ven...   
1  Fortune Barishal secured a spot in the Banglad...   
2  The Ministry of Home Affairs (MHA) could notif...   
3  Fortune Barishal secured a seven-wicket victor...   
4  Bangladesh Premier League 2024: The Bangladesh...   

                                     content_summary title_translation  \
0  Comilla Victorians vs Fortune Barishal Banglad...              None   
1  Here's a look at the highest run-scorers and w...              None   
2  The CAA rules would ensure the processing of I...              None   
3  Here's a look at the highest run-scorers and w...              None   
4  Here is the list of updated points table, high...              None   

  content_translation summary translation           author     country  \
0                None                None  ABP News Bureau  bangladesh   
1                None                None  ABP News Bureau  bangladesh   
2                None                None  ABP News Bureau  bangladesh   
3                None                None  ABP News Bureau  bangladesh   
4                None                None  ABP News Bureau  bangladesh   

     source_localtime bangladesh_localtime  
0 2024-02-29 21:26:00  2024-02-29 21:56:00  
1 2024-02-29 05:06:00  2024-02-29 05:36:00  
2 2024-02-28 02:34:00  2024-02-28 03:04:00  
3 2024-02-27 13:03:00  2024-02-27 13:33:00  
4 2024-02-23 13:41:00  2024-02-23 14:11:00